In [5]:
from keras.models import *

In [6]:
main_model = load_model('my_model')
main_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, None, None, 1 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [7]:
expo_model = load_model('exposure_model')
expo_model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 30, 30, 8)         608       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 15, 15, 8)         0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 15, 15, 8)         0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 7, 7, 4)           292       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 3, 3, 4)           0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 3, 3, 4)           0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 1, 1, 32)        